### Análise de Sentimentos com NLTK

A análise de sentimentos é uma tarefa comum da PNL, que envolve a classificação de textos ou partes de textos em um sentimento predefinido.

#### Importação das Bibliotecas

In [1]:
import nltk
from nltk.corpus import twitter_samples
from nltk.tag import pos_tag
from nltk.stem.wordnet import WordNetLemmatizer 
from nltk.corpus import stopwords
from nltk import FreqDist
from nltk.tokenize import word_tokenize  
from nltk import classify
from nltk import NaiveBayesClassifier


import re, string, random

In [2]:
nltk.download('twitter_samples')
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package twitter_samples to /root/nltk_data...
[nltk_data]   Unzipping corpora/twitter_samples.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

In [3]:
def remove_noise(tweet_tokens, stop_words = ()):
    cleaned_tokens = []
    for token, tag in pos_tag(tweet_tokens):
        token = re.sub('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+#]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+','', token)
        token = re.sub("(@[A-Za-z0-9_]+)","", token)
        if tag.startswith("NN"):
            pos = 'n'
        elif tag.startswith('VB'):
            pos = 'v'
        else:
            pos = 'a'
        lemmatizer = WordNetLemmatizer()
        token = lemmatizer.lemmatize(token, pos)
        if len(token) > 0 and token not in string.punctuation and token.lower() not in stop_words:
            cleaned_tokens.append(token.lower())
    return cleaned_tokens

In [4]:
def get_all_words(cleaned_tokens_list):
    for tokens in cleaned_tokens_list:
        for token in tokens:
            yield token 

In [5]:
def get_tweets_for_model(cleaned_tokens_list):
    for tweet_tokens in cleaned_tokens_list:
        yield dict([token, True] for token in tweet_tokens) 

#### Leitura dos dados

In [6]:
positive_tweets = twitter_samples.strings('positive_tweets.json')
negative_tweets = twitter_samples.strings('negative_tweets.json')
text = twitter_samples.strings('tweets.20150430-223406.json')

In [7]:
print(f'Tamanho: {len(positive_tweets)}')
print(f'Tamanho: {len(negative_tweets)}')
print(f'Tamanho: {len(text)}')

Tamanho: 5000
Tamanho: 5000
Tamanho: 20000


#### Tokenização

In [8]:
positive_tweets_tokens = twitter_samples.tokenized('positive_tweets.json')
negative_tweets_tokens = twitter_samples.tokenized('negative_tweets.json')
tweet_tokens = twitter_samples.tokenized('tweets.20150430-223406.json')

#### Normalização e Limpeza dos dados

In [9]:
stop_words = stopwords.words('english') 
positive_cleaned_tokens_list = []
negative_cleaned_tokens_list = []
for tokens in positive_tweets_tokens:
    positive_cleaned_tokens_list.append(remove_noise(tokens, stop_words))
for tokens in negative_tweets_tokens:
    negative_cleaned_tokens_list.append(remove_noise(tokens, stop_words))

#### Preparação dos dados para o modelo

In [10]:
positive_tokens_for_model = get_tweets_for_model(positive_cleaned_tokens_list)
negative_tokens_for_model = get_tweets_for_model(negative_cleaned_tokens_list)

#### Divisão do conjunto de dados para treinamento e teste do modelo

In [11]:
positive_dataset = [(tweet_dict, "Positive")for tweet_dict in positive_tokens_for_model] 
negative_dataset = [(tweet_dict, "Negative")for tweet_dict in negative_tokens_for_model]
dataset = positive_dataset + negative_dataset 
random.shuffle(dataset)

train_data = dataset[:7000]
test_data = dataset[7000:]

#### Criando e testando o modelo

In [13]:
classifier = NaiveBayesClassifier.train(train_data) 
print("Accuracy is:", classify.accuracy(classifier, test_data)) 
print(classifier.show_most_informative_features(10))

Accuracy is: 0.997
Most Informative Features
                      :) = True           Positi : Negati =    986.8 : 1.0
                follower = True           Positi : Negati =     31.9 : 1.0
                     sad = True           Negati : Positi =     24.5 : 1.0
                    glad = True           Positi : Negati =     20.7 : 1.0
                   enjoy = True           Positi : Negati =     14.2 : 1.0
                   didnt = True           Negati : Positi =     11.8 : 1.0
               goodnight = True           Positi : Negati =     11.5 : 1.0
              definitely = True           Positi : Negati =     10.8 : 1.0
                    miss = True           Negati : Positi =     10.8 : 1.0
                 welcome = True           Positi : Negati =     10.0 : 1.0
None
